In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats
import pickle as pkl
import scipy.interpolate

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')
RAWDATAPATH = os.path.join(PATH, 'sim_data')

"""
Date: 6/25/2021
Description: template
"""

gradB_file = '210830_gradB_motion_Trap_V00_00_10.pkl'

with open(os.path.join(RAWDATAPATH, gradB_file), 'rb') as infile:
    gradB_data = pkl.load(infile)

dataset_file = '210729_df_84/210726_df_84_1.h5'
h5datafile = h5py.File(os.path.join(PATH, 'sim_data/datasets', dataset_file), 'r')

var = 1.38e-23 * 10 * 50 * 200e6

def SumDataLocust(array_data, xpos, ypos, r=0.1, f=25.86e9, antispiral=False, nch=60):
    
    angles = np.arange(0, nch, 1) * 360 // 60
    xx, yy = np.meshgrid(xpos, ypos)
    xx, yy = np.expand_dims(xx, axis=0).repeat(angles.size, axis=0), np.expand_dims(yy, axis=0).repeat(angles.size, axis=0)

    xantenna = (r * np.cos(np.radians(-1 * angles))).reshape((angles.size, 1, 1)).repeat(xpos.size, axis=1).repeat(xpos.size, axis=2)
    yantenna = (r * np.sin(np.radians(-1 * angles))).reshape((angles.size, 1, 1)).repeat(ypos.size, axis=1).repeat(ypos.size, axis=2)

    xdist = xantenna - xx
    ydist = yantenna - yy
    
    dist_grid = np.sqrt(xdist**2 + ydist**2)
    
    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    if antispiral:
        antispiral_phase = np.exp(1j * np.radians( 1 * angles)).reshape((angles.size, 1)).repeat(array_data.shape[-1], axis=-1)
        return np.matmul(beamforming_phases.T, antispiral_phase * array_data)
    else:
        return np.matmul(beamforming_phases.T, array_data)

    
def ShiftSignalPosition(array_data, x_pos, y_pos, r=0.1, f=25.86e9, nch=60):
    
    angles = np.arange(0, nch, 1) * 360 // 60
    xantenna = r * np.cos(np.radians(angles))
    yantenna = r * np.sin(np.radians(angles))
    
    antenna_phase = 2 * np.pi * np.sqrt((xantenna - x_pos) ** 2 + (yantenna - y_pos) ** 2) / (3e8 / f)
    
    relative_amplitude = (r ** 2) / ((xantenna - x_pos) ** 2 + (yantenna - y_pos) ** 2)
    
    correction = relative_amplitude * np.exp(1j * antenna_phase)
    
    corrected_array_data = correction.reshape((1, correction.size, 1)).repeat(array_data.shape[0], axis=0).repeat(array_data.shape[-1], axis=-1) * array_data
    
    return corrected_array_data


def ShiftSignalPositionGradB(array_data, angle, x_pos, y_pos, interpolator, r=0.1, f=25.86e9, nch=60, fsamp=200e6):
    
    nslice = array_data.shape[0]
    nsamp = array_data.shape[-1]
    nch = array_data.shape[1]
    t = (np.arange(0, nslice * nsamp, 1) / fsamp).reshape(nslice, 1, nsamp).repeat(nch, axis=1)
    
    angles_antenna = np.arange(0, nch, 1) * 360 // 60
    xantenna = r * np.cos(np.radians(angles_antenna)).reshape((1, nch, 1)).repeat(nslice, axis=0).repeat(nsamp, axis=-1)
    yantenna = r * np.sin(np.radians(angles_antenna)).reshape((1, nch, 1)).repeat(nslice, axis=0).repeat(nsamp, axis=-1)
    
    rad_pos = np.sqrt(x_pos ** 2 + y_pos ** 2)
    gradb_freq = 1 * interpolator(rad_pos, angle)
    
    gradb_angle_pos = gradb_freq * t + np.arctan2(y_pos, x_pos)# initial + time_dep
    
    x_t = rad_pos * np.cos(gradb_angle_pos)
    y_t = rad_pos * np.sin(gradb_angle_pos)
    
    antenna_phase = 2 * np.pi * np.sqrt((xantenna - x_t) ** 2 + (yantenna - y_t) ** 2) / (3e8 / f)
    
    relative_amplitude = (r ** 2) / ((xantenna - x_t) ** 2 + (yantenna - y_t) ** 2)
    
    correction = relative_amplitude * np.exp(1j * antenna_phase)
    
    corrected_array_data = correction * array_data
    
    return corrected_array_data

def CreateGradBInterpolator(path):
    
    with open(os.path.join(path), 'rb') as infile:
        gradB_data = pkl.load(infile)
        
    gradb_pd = pd.DataFrame(gradB_data, columns = ['r', 'angle', 'freq'])
        
    angles = np.sort(np.unique(gradB_data[:, 1]))
    rads = np.sort(np.unique(gradB_data[:, 0]))
    rad_grid, angle_grid = np.meshgrid(rads, angles)

    frequency_grid = np.zeros(rad_grid.shape)

    for i in range(frequency_grid.shape[0]):
        for j in range(frequency_grid.shape[1]):
            frequency_grid[i, j] = gradb_pd[(gradb_pd['r'] == rad_grid[i, j]) & (gradb_pd['angle'] == angle_grid[i, j])]['freq']


    gradb_interpolation = scipy.interpolate.interp2d(rads, angles, frequency_grid, kind='cubic')
    
    return gradb_interpolation


In [ ]:
h5file = h5py.File(os.path.join(SIMDATAPATH, filename))

In [ ]:
signal = h5file['signal']['6'][:]

In [ ]:
x_grid = np.linspace(-0.03, 0.03, 101)
y_grid = np.linspace(-0.03, 0.03, 101)

summed_signal = SumDataLocust(signal[0, :, :], x_grid, y_grid, antispiral=True, f=25.86e9)

In [ ]:
plt.imshow(abs(summed_signal.sum(axis=-1)**2).T)

In [ ]:
for i, item in enumerate(h5file['signal']['6'].attrs.items()):
    print(item)

In [ ]:
interpolator = CreateGradBInterpolator(os.path.join(RAWDATAPATH, gradB_file))

In [ ]:
#shifted_signal = ShiftSignalPosition(signal, 0.01, 0.0)
shifted_signal = ShiftSignalPositionGradB(signal, 89.02, 0.01, 0, interpolator)

In [ ]:
x_grid = np.linspace(-0.03, 0.03, 101)
y_grid = np.linspace(-0.03, 0.03, 101)

summed_shifted_signal = SumDataLocust(shifted_signal[0, :, :], x_grid, y_grid, antispiral=True, f=25.86e9)

In [ ]:
summed_shifted_signal.shape

In [ ]:
plt.imshow(abs(summed_shifted_signal.sum(axis=-1)**2).T)